# NWB Tutorial - Extracellular Electrophysiology

## Introduction
In this tutorial, we will create an NWB file for a hypothetical extracellular electrophysiology experiment with a freely moving animal. The types of data we will convert are:
- Subject information (species, strain, age, etc.) 
- Animal position
- Trials
- Raw data
- LFP
- Spike times

## Installing PyNWB
First, install PyNWB using pip or conda. You will need Python 3.5+ installed.
- `pip install pynwb`
- `conda install -c conda-forge pynwb`

## Set up the NWB file
An NWB file represents a single session of an experiment. Each file must have a session description, identifier, and session start time. Importantly, the session start time is the reference time for all timestamps in the file. For example, an event with a timestamp of 0 in the file means the event occurred exactly at the session start time. 

Create a new `NWBFile` object with those and additional metadata. For all constructors in PyNWB, we recommend using keyword arguments for clarity.

In [ ]:
from pynwb import NWBFile
from datetime import datetime
from dateutil import tz

session_start_time = datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz('US/Pacific'))

nwbfile = NWBFile(
    session_description='Mouse exploring an open field',
    identifier='Mouse5_Day3',
    session_start_time=session_start_time,
    session_id='session_1234',                                # optional
    experimenter='My Name',                                   # optional
    lab='My Lab Name',                                        # optional
    institution='University of My Institution',               # optional
    related_publications='DOI:10.1016/j.neuron.2016.12.011'   # optional
)
print(nwbfile)

## Subject information
Create a `Subject` object to store information about the experimental subject, such as age, species, genotype, sex, and a description. Then set `nwbfile.subject` to the new `Subject` object.

<img src="images/Subject.svg" width="150">

Each of these fields is free-form text, so any values will be valid, but we recommend these values follow particular conventions to help software tools interpret the data:
- For age, we recommend using the [ISO 8601 Duration format](https://en.wikipedia.org/wiki/ISO_8601#Durations), e.g., "P90D" for 90 days old
- For species, we recommend using the formal latin binomal name, e.g., "*Mus musculus*", "*Homo sapiens*"
- For sex, we recommend using "F" (female), "M" (male), "U" (unknown), and "O" (other)

In [ ]:
from pynwb.file import Subject

nwbfile.subject = Subject(
    subject_id='001',
    age='P90D', 
    description='mouse 5',
    species='Mus musculus', 
    sex='M'
)

## SpatialSeries and Position
PyNWB contains classes that are specialized for different types of data. To store the spatial position of a subject, we will use the `SpatialSeries` and `Position` classes. 

`SpatialSeries` is a subclass of `TimeSeries`. `TimeSeries` is a common base class for measurements sampled over time, and provides fields for data and time (regularly or irregularly sampled).

<img src="images/SpatialSeries.svg" width="200">

Create a `SpatialSeries` object named `'SpatialSeries'` with some fake data.

In [ ]:
import numpy as np
from pynwb.behavior import SpatialSeries

# create fake data with shape (50, 2)
# the first dimension should always represent time
position_data = np.array([np.linspace(0, 10, 50),
                          np.linspace(0, 8, 50)]).T
position_timestamps = np.linspace(0, 50) / 200

spatial_series_obj = SpatialSeries(
    name='SpatialSeries', 
    description='(x,y) position in open field',
    data=position_data,
    timestamps=position_timestamps,
    reference_frame='(0,0) is bottom left corner'
)

You can print the `SpatialSeries` object to view its contents.

In [ ]:
print(spatial_series_obj)

To help data analysis and visualization tools know that this `SpatialSeries` object represents the position of the subject, store the `SpatialSeries` object inside of a `Position` object, which can hold one or more `SpatialSeries` objects.

<img src="images/Position.svg" width="450">

In [ ]:
from pynwb.behavior import Position

position_obj = Position(spatial_series=spatial_series_obj)  # name is set to 'Position' by default

### Behavior Processing Module

NWB differentiates between raw, *acquired data*, which should never change, and *processed data*, which are the results of preprocessing algorithms and could change. Let's assume that the subject's position was computed from a video tracking algorithm, so it would be classified as processed data. Since processed data can be diverse, NWB allows us to create processing modules, which are like folders, to store related processed data. 

Create a processing module called "behavior" for storing behavioral data in the `NWBFile` and add the `Position` object to the processing module.

In [ ]:
behavior_module = nwbfile.create_processing_module(
    name='behavior', 
    description='processed behavioral data'
)
behavior_module.add(position_obj)

<img src="images/Behavior.svg" width="600">

## Write to file

Now, write the NWB file that we have built so far.

In [ ]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('ecephys_tutorial.nwb', 'w') as io:
    io.write(nwbfile)

## Read from an NWB file

We can now read the file and print it to inspect its contents. 

We can also print the SpatialSeries data that we created by referencing the names of the objects in the hierarchy that contain it. We can access a processing module by indexing `nwbfile.processing` with the name of the processing module, which in our case is "behavior". 

Then, we can access the `Position` object inside of the "behavior" processing module by indexing it with the name of the `Position` object. The default name of `Position` objects is "Position". 

Finally, we can access the `SpatialSeries` object inside of the `Position` object by indexing it with the name of the `SpatialSeries` object, which we named `'SpatialSeries'`.

In [ ]:
with NWBHDF5IO('ecephys_tutorial.nwb', 'r') as io:
    read_nwbfile = io.read()
    print(read_nwbfile.processing['behavior'])
    print(read_nwbfile.processing['behavior']['Position'])
    print(read_nwbfile.processing['behavior']['Position']['SpatialSeries'])

We can also use the [HDFView](https://www.hdfgroup.org/downloads/hdfview/) tool to inspect the resulting NWB file.

<img src="images/position_hdfview.png" width="400">

## Trials

Trials are stored in a `TimeIntervals` object which is a subclass of `DynamicTable`. `DynamicTable` objects are used to store tabular metadata throughout NWB, including for trials, electrodes, and sorted units. They offer flexibility for tabular data by allowing required columns, optional columns, and custom columns not defined in the standard.

<img src="images/Trials.svg" width="300">

The trials DynamicTable can be thought of as a table with this structure:

<img src="images/trials_example.png" width="400">

We can add custom, user-defined columns to the trials table to hold data and metadata specific to this experiment or session. Continue adding to our `NWBFile` by creating a new column for the trials table named `'correct'`, which will be a boolean array.

In [ ]:
nwbfile.add_trial_column(name='correct', description='whether the trial was correct')
nwbfile.add_trial(start_time=1.0, stop_time=5.0, correct=True)
nwbfile.add_trial(start_time=6.0, stop_time=10.0, correct=False)

We can view and inspect the trials table in tabular form by converting it to a pandas dataframe.

In [ ]:
df = nwbfile.trials.to_dataframe()
df

## Extracellular electrophysiology

## Electrodes table
In order to store extracellular electrophysiology data, you first must create an electrodes table describing the electrodes that generated this data. Extracellular electrodes are stored in an `electrodes` table, which is also a `DynamicTable`. The `electrodes` table has several required fields: x, y, z, impedence, location, filtering, and electrode group.

<img src="images/Electrodes.png" width="500">

Since this is a DynamicTable, we can add additional metadata fields. We will be adding a "label" column to the table.
Use the following code to add electrodes for an array with 4 shanks and 3 channels per shank.

In [ ]:
nwbfile.add_electrode_column(name='label', description='label of electrode')

nshanks = 4;
nchannels_per_shank = 3;
electrode_counter = 0
device = nwbfile.create_device(
    name='array',
    description='the best array',
    manufacturer='Probe Company 9000'
)
for ishank in range(nshanks):
    # create an electrode group for this shank
    electrode_group = nwbfile.create_electrode_group(
       name='shank{}'.format(ishank),
       description='electrode group for shank {}'.format(ishank),
       device=device,
       location='brain area'
    )
    # add electrodes to the electrode table
    for ielec in range(nchannels_per_shank):
        nwbfile.add_electrode(
            x=5.3, y=1.5, z=8.5, imp=np.nan,
            location='unknown', 
            filtering='unknown',
            group=electrode_group,
            label='shank{}elec{}'.format(ishank, ielec)
        )
        electrode_counter += 1

Like for the trials table, we can view the electrodes table in tabular form by converting it to a pandas dataframe.

In [ ]:
df = nwbfile.electrodes.to_dataframe()
df

## Links
In the above loop, we created `ElectrodeGroup` objects in the `NWBFile`, and when we added an electrode to the `NWBFile`, we passed in the `ElectrodeGroup` object for the required `'group'` argument. This creates a reference from the `electrodes` table to individual `ElectrodeGroup` objects, one per row (electrode).

## ElectricalSeries and DynamicTableRegion
Raw voltage data and LFP data are stored in `ElectricalSeries` objects. `ElectricalSeries` is a subclass of `TimeSeries` specialized for voltage data. In order to create our `ElectricalSeries` objects, we will need to reference a set of rows in the `electrodes` table to indicate which electrodes were recorded. We will do this by creating a `DynamicTableRegion`, which is a type of link that allows you to reference specific rows of a `DynamicTable`, such as the `electrodes` table, by row indices.

Create a `DynamicTableRegion` that references all rows of the `electrodes` table.

In [ ]:
all_table_region = nwbfile.create_electrode_table_region(
    region=list(range(electrode_counter)),  # reference row indices 0 to N-1
    description='all electrodes'
)

## Raw data

Now create an `ElectricalSeries` object to hold raw data collected during the experiment, passing in this `DynamicTableRegion` reference to all rows of the `electrodes` table.

<img src="images/ElectricalSeries.png" width="800">

In [ ]:
from pynwb.ecephys import ElectricalSeries

raw_data = np.random.randn(50, 4)
raw_elec_series = ElectricalSeries(
    name='ElectricalSeries', 
    data=raw_data, 
    electrodes=all_table_region, 
    starting_time=0.,
    rate=20000.
)

NWB organizes data into different groups depending on the type of data. Groups can be thought of as folders within the file. Here are some of the groups within an NWB file and the types of data they are intended to store:
- **acquisition**: raw, acquired data that should never change
- **processing**: processed data, typically the results of preprocessing algorithms and could change
- **analysis**: results of data analysis
- **stimuli**: stimuli used in the experiment (e.g., images, videos, light pulses)

Since this `ElectricalSeries` represents raw data from the data acquisition system, add it to the acquisition group of the NWB file.

In [ ]:
nwbfile.add_acquisition(raw_elec_series)

## LFP

Now create an `ElectricalSeries` object to hold LFP data collected during the experiment, again passing in the `DynamicTableRegion` reference to all rows of the `electrodes` table.

In [ ]:
lfp_data = np.random.randn(50, 4)
lfp_elec_series = ElectricalSeries(
    name='ElectricalSeries', 
    data=lfp_data, 
    electrodes=all_table_region, 
    starting_time=0.,
    rate=200.
)

To help data analysis and visualization tools know that this `ElectricalSeries` object represents LFP data, store the `ElectricalSeries` object inside of an `LFP` object. This is analogous to how we stored the `SpatialSeries` object inside of a `Position` object earlier.

<img src="images/LFP.png" width="800">

In [ ]:
from pynwb.ecephys import LFP

lfp = LFP(electrical_series=lfp_elec_series)

Unlike the raw data, which we put into the acquisition group of the NWB file, LFP data is typically considered processed data because the raw data was filtered and downsampled to generate the LFP. 

Create a processing module named `'ecephys'` and add the `LFP` object to it. This is analogous to how we stored the `Position` object in a processing module named `'behavior'` earlier.

In [ ]:
ecephys_module = nwbfile.create_processing_module(
    name='ecephys', 
    description='processed extracellular electrophysiology data'
)
ecephys_module.add(lfp)

## Spike Times
Spike times are stored in the `Units` table, which is another subclass of `DynamicTable`. We can add columns to the `Units` table just like we did for the electrodes and trials tables. 

Generate some random spike data and populate the `Units` table using `nwbfile.add_unit`. Then display the `Units` table as a pandas dataframe.

In [ ]:
nwbfile.add_unit_column(name='quality', description='sorting quality')

poisson_lambda = 20
firing_rate = 20
n_units = 10
for n_units_per_shank in range(n_units):
    n_spikes = np.random.poisson(lam=poisson_lambda)
    spike_times = np.round(np.cumsum(np.random.exponential(1/firing_rate, n_spikes)), 5)
    nwbfile.add_unit(spike_times=spike_times, quality='good', waveform_mean=[1., 2., 3., 4., 5.])

df = nwbfile.units.to_dataframe()
df

## Write the file

In [ ]:
with NWBHDF5IO('ecephys_tutorial.nwb', 'w') as io:
    io.write(nwbfile)

## Read the NWB file

We can access the raw data by indexing `nwbfile.acquisition` with the name of the `ElectricalSeries`, which we named `'ElectricalSeries'`. 

We can also access the LFP data by indexing `nwbfile.processing` with the name of the processing module, "ecephys". Then, we can access the `LFP` object inside of the "ecephys" processing module by indexing it with the name of the `LFP` object. The default name of `LFP` objects is "LFP". 

Finally, we can access the `ElectricalSeries` object inside of the `LFP` object by indexing it with the name of the `ElectricalSeries` object, which we named `'ElectricalSeries'`.

In [ ]:
with NWBHDF5IO('ecephys_tutorial.nwb', 'r') as io:
    read_nwbfile = io.read()
    print(read_nwbfile.acquisition['ElectricalSeries'])
    print(read_nwbfile.processing['ecephys'])
    print(read_nwbfile.processing['ecephys']['LFP'])
    print(read_nwbfile.processing['ecephys']['LFP']['ElectricalSeries'])

## Reading NWB data

Data arrays are read passively from the file. Calling the `data` attribute on a `TimeSeries` such as an `ElectricalSeries` or `SpatialSeries` does not read the data values, but presents an `h5py` object that can be indexed to read data. You can use the `[:]` operator to read the entire data array into memory.

Load and print all the `data` values of the `ElectricalSeries` object representing the LFP data.

In [ ]:
with NWBHDF5IO('ecephys_tutorial.nwb', 'r') as io:
    read_nwbfile = io.read()
    print(read_nwbfile.processing['ecephys']['LFP']['ElectricalSeries'].data[:])

## Accessing data regions
It is often preferable to read only a portion of the data. To do this, index or slice into the `data` attribute just like if you were indexing or slicing a numpy array.

The following code prints elements 0:10 in the first dimension (time) and 0:3 in the second dimension (electrodes) from the LFP data we have written.

Accessing data from a `DynamicTable` is similar: `read_nwbfile.units['spike_times'][0]` reads only the spike times from the 0th unit (the value at the 0th row and column named 'spike_times' of the `Units` table).

In [ ]:
with NWBHDF5IO('ecephys_tutorial.nwb', 'r') as io:
    read_nwbfile = io.read()

    print('section of lfp:')
    print(read_nwbfile.processing['ecephys']['LFP']['ElectricalSeries'].data[:10,:3])
    print('')
    print('spike times from 0th unit:')
    print(read_nwbfile.units['spike_times'][0])

# Learn more!

## Python tutorials
### See our tutorials for more details about your data type:
* [Extracellular electrophysiology](https://pynwb.readthedocs.io/en/stable/tutorials/domain/ecephys.html#sphx-glr-tutorials-domain-ecephys-py)
* [Calcium imaging](https://pynwb.readthedocs.io/en/stable/tutorials/domain/ophys.html#sphx-glr-tutorials-domain-ophys-py)
* [Intracellular electrophysiology](https://pynwb.readthedocs.io/en/stable/tutorials/domain/icephys.html#sphx-glr-tutorials-domain-icephys-py)

### Check out other tutorials that teach advanced NWB topics:
* [Iterative data write](https://pynwb.readthedocs.io/en/stable/tutorials/general/iterative_write.html#sphx-glr-tutorials-general-iterative-write-py)
* [Extensions](https://pynwb.readthedocs.io/en/stable/tutorials/general/extensions.html#sphx-glr-tutorials-general-extensions-py)
* [Advanced HDF5 I/O](https://pynwb.readthedocs.io/en/stable/tutorials/general/advanced_hdf5_io.html#sphx-glr-tutorials-general-advanced-hdf5-io-py)


## MATLAB tutorials
* [Extracellular electrophysiology](https://neurodatawithoutborders.github.io/matnwb/tutorials/html/ecephys.html)
* [Calcium imaging](https://neurodatawithoutborders.github.io/matnwb/tutorials/html/ophys.html)
* [Intracellular electrophysiology](https://neurodatawithoutborders.github.io/matnwb/tutorials/html/icephys.html)
